---
# 0. 라이브러리
----

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# 깃 허브에 있는 데이텃뎃 다운로드를 위한 깃 클론
!git clone https://github.com/arijitx/Fewshot-Learning-with-BERT.git

# dataset이 있는 경로 지정
%cd /content/Fewshot-Learning-with-BERT/data

fatal: destination path 'Fewshot-Learning-with-BERT' already exists and is not an empty directory.
/content/Fewshot-Learning-with-BERT/data


In [6]:
!pip install torch pytorch-lightning
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[참고 사이트](https://uvadlc-notebooks.readthedocs.io/en/latest/tutorial_notebooks/tutorial16/Meta_Learning.html)

In [7]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from tqdm.notebook import tqdm
from torchvision import transforms, datasets
from sklearn.model_selection import train_test_split

from PIL import Image
from pprint import pprint
import easydict

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') # GPU 할당

import warnings
warnings.filterwarnings(action='ignore')

import seaborn as sns 
import pandas as pd
import os
import random
import json
from collections import defaultdict
from statistics import mean, stdev
from copy import deepcopy


import matplotlib.pyplot as plt
plt.set_cmap('cividis')
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # For export
import matplotlib
matplotlib.rcParams['lines.linewidth'] = 2.0

# transformers
from transformers import BertTokenizer, BertModel
import math
import pdb

# tensorboard
%load_ext tensorboard

# Pytorch Lightning
import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [8]:
import easydict

args = easydict.EasyDict({
  'lr' : 2e-4,
  'max_len' : 110,
  'epoch' : 50,
  'N_WAY' : 5,
  'K_SHOT' : 4
})

---
# 2. Data
----

- ATIS 데이터셋을 통해 train / validation / test 셋을 구성

In [9]:
# train
text = pd.read_csv('./ATIS/train/seq.in', names = ['text'])
label = pd.read_csv('./ATIS/train/label', names = ['label'])
train_data = pd.concat([text,label], axis = 1)

# valid
text = pd.read_csv('./ATIS/valid/seq.in', names = ['text'])
label = pd.read_csv('./ATIS/valid/label', names = ['label'])
valid_data = pd.concat([text,label], axis = 1)

# test
text = pd.read_csv('./ATIS/test/seq.in', names = ['text'])
label = pd.read_csv('./ATIS/test/label', names = ['label'])
test_data = pd.concat([text,label], axis = 1)

print(train_data.shape)
print(valid_data.shape)
print(test_data.shape)

(4478, 2)
(500, 2)
(893, 2)


- label 종류 확인

In [10]:
train_label = set(train_data['label'])
valid_label = set(valid_data['label'])
test_label = set(test_data['label'])

In [11]:
print('train_set label 개수 : ', len(train_label))
print('valid_set label 개수 : ', len(valid_label))
print('test_set label 개수 : ', len(test_label))

train_set label 개수 :  21
valid_set label 개수 :  16
test_set label 개수 :  20


In [12]:
uni_label = train_label | valid_label | test_label
label_to_index = {i : idx for idx, i in enumerate(uni_label)}
index_to_label = {idx : i for idx, i in enumerate(uni_label)}

In [13]:
label_to_index

{'atis_flight#atis_airfare': 0,
 'atis_day_name': 1,
 'atis_airport': 2,
 'atis_airfare#atis_flight': 3,
 'atis_flight_time': 4,
 'atis_ground_fare': 5,
 'atis_flight': 6,
 'atis_airline#atis_flight_no': 7,
 'atis_airfare': 8,
 'atis_ground_service#atis_ground_fare': 9,
 'atis_ground_service': 10,
 'atis_flight_no': 11,
 'atis_airline': 12,
 'atis_aircraft': 13,
 'atis_airfare#atis_flight_time': 14,
 'atis_capacity': 15,
 'atis_city': 16,
 'atis_quantity': 17,
 'atis_restriction': 18,
 'atis_cheapest': 19,
 'atis_flight_no#atis_airline': 20,
 'atis_abbreviation': 21,
 'atis_meal': 22,
 'atis_distance': 23,
 'atis_aircraft#atis_flight#atis_flight_no': 24,
 'atis_flight#atis_airline': 25}

In [14]:
def trans_label(train_data):
  for i in range(len(train_data)):
    train_data['label'][i] = label_to_index[train_data['label'][i]]
  return train_data

In [15]:
trans_label(train_data)
trans_label(valid_data)
trans_label(test_data)

,text,label
0,i would like to find a flight from charlotte t...,6
1,on april first i need a ticket from tacoma to ...,8
2,on april first i need a flight going from phoe...,6
3,i would like a flight traveling one way from p...,6
4,i would like a flight from orlando to salt lak...,6
...,...,...
888,please find all the flights from cincinnati to...,6
889,find me a flight from cincinnati to any airpor...,6
890,i'd like to fly from miami to chicago on ameri...,6
891,i would like to book a round trip flight from ...,6


In [16]:
value = pd.concat([train_data, test_data, valid_data], ignore_index = True)
value

,text,label
0,i want to fly from baltimore to dallas round trip,6
1,round trip fares from baltimore to philadelphi...,8
2,show me the flights arriving on baltimore on j...,6
3,what are the flights which depart from san fra...,6
4,which airlines fly from boston to washington d...,12
...,...,...
5866,pm flights dallas to atlanta,6
5867,information on flights from baltimore to phila...,6
5868,what flights from atlanta to st. louis on tues...,6
5869,show me ground transportation in san francisco,10


In [17]:
unique_label_dict = {}
# 레이블별 개수 확인
for i in range(len(value)):
  if value['label'][i] in unique_label_dict.keys():
    unique_label_dict[value['label'][i]] += 1
  else:
    unique_label_dict[value['label'][i]] = 1

# 10개 이하인 레이블 확인
qq = []
for k, v in unique_label_dict.items():
  if v <= 10:
    qq.append(k)    

# 10개 이하인 레이블 삭제
for i in qq:
  del unique_label_dict[i]

In [18]:
dataset = pd.DataFrame()
for i in unique_label_dict.keys():
  dataset = pd.concat([dataset,value[value['label'] == i]])

dataset = dataset.reset_index(drop = True)  
unique_label_dict

{6: 4298,
 8: 471,
 12: 195,
 10: 291,
 17: 54,
 16: 25,
 0: 33,
 21: 180,
 13: 90,
 23: 30,
 5: 25,
 15: 37,
 4: 55,
 22: 12,
 11: 20,
 2: 38}

In [19]:
split_data, test_data = dataset[: -187], dataset[-187:]
train_data, valid_data = split_data[:-358], split_data[-358:]

train_data = train_data.sample(frac = 1).reset_index(drop = True)
valid_data = valid_data.sample(frac = 1).reset_index(drop = True)
test_data = test_data[:-1].sample(frac = 1).reset_index(drop = True)

print(train_data.shape) # 5개 클래스 
print(valid_data.shape) # 5개 클래스
print(test_data.shape) # 6개 클래스

(5309, 2)
(358, 2)
(186, 2)


---
# 3. Preprocessing
---

In [20]:
class BertDataset(Dataset):
  def __init__(self, data):
    self.sentence1 = []
    self.label = []
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

    for meta in data['text']:
      # BERT -> max 512 length
      self.sentence1.append(tokenizer(meta, max_length=args.max_len, padding='max_length', return_tensors='pt'))
      
    for meta in data['label']:
      self.label.append(meta)

    self.label = torch.tensor(self.label)

  def __len__(self):
    return len(self.sentence1)

  def __getitem__(self, idx):
    return self.sentence1[idx], self.label[idx]

In [21]:
BertDataset(train_data).__getitem__(0)

({'input_ids': tensor([[ 101, 2054, 2024, 7176, 7599, 2013, 3190, 2046, 5862, 2006, 2238, 3587,
           102,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [22]:
train_set = BertDataset(train_data)

valid_set = BertDataset(valid_data)

---
# 4. Data Sampling
----

In [23]:
class FewShotBatchSampler():

  def __init__(self, dataset_targets, N_way, K_shot,
               include_query = False,
               shuffle = True,
               shuffle_once = False):
    super().__init__()
  
    '''dataset_targets = 데이터 요소 레이블의 tensor
        N_way = 배치당 샘플링할 클래스 수
        K-shot = 배치에서 클래스 당 샘플링할 예제 수
        include_query = True인 경우 N_way * K-shot * 2 크기의 배치를 반환
          sup 및 qry 세트에 대해 동일한 클래스와 별개의 예제를 샘플링하는 구현을 단순화
        shuffle = True 이면 예제 및 클래스가 각 항목에서 새로 shffle
        shuffle_once = True 이면 예제와 클래스가 한 번씩 셔플 됨'''

    self.dataset_targets = dataset_targets
    self.N_way = N_way
    self.K_shot = K_shot
    self.shuffle = shuffle
    self.include_query = include_query
    if self.include_query :
      self.K_shot *= 2
    # self.batch_size = self.N_way * self.K_shot

    '''batch_size를 N_way와 K_shot를 곱한 값으로 하는 이유는??
          
          N-way K-shot classification 문제를 해결하기 위해
          
          N-way K-shot문제란 N 개의 클래스 중 K개의 샘플만 사용하여 학습하는 분류 문제인데 
          N = 클래스의 수 / K = 각 클래스 마다 사용되는 샘플의 개수 
          N_way = 한 배치당 몇 개의 클래스르 사용할지 결정하는 변수
          K_shot = 각 클래스 당 몇 개의 샘플을 사용할지 결정하는 변수
          
          예를 들어  N_way = 5, K_shot = 1인 경우, 한 배치는 5개의 클래스를 가지며, 각 클래스당 1개의 샘플을 사용하여
          총 5 * 1 = 5개의 샘플이 한 배치를 이루게 된다.
          따라서 N_way와 K_shot를 곱하는 이뉴는 
          한 배치에서 사용할 이미지의 개수를 결정하기 위해서 이다.'''
    
    ############################# 클래스별 예제 구성 #############################
    # 고유한 클래스 라벨 추출
    self.classes = torch.unique(self.dataset_targets).tolist()
    
    # 클래스 총 개수 추출
    self.num_classes = len(self.classes)

    # 클래스 별 데이터셋의 인덱스
    self.indices_per_class = {}
    
    # 클래스 별 데이터셋의 K_shot 배치 개수
    self.batches_per_class = {} 

    # 각 클래스 별로 데이터셋의 인덱스를 추출
    for i in self.classes:
      self.indices_per_class[i] = torch.where(self.dataset_targets == i)[0]
      
      # 해당 클래스에서 만들 수 있는 k-shot 배치의 개수를 의미
      self.batches_per_class[i] = self.indices_per_class[i].shape[0] // self.K_shot
    ##########################################################################

    ############################# batch 당 N 개의 클래스를 선택하는 클래스 목록 구성 #############################
    # 각 클래스 마다 K-shot개의 이미지로 이루어진 배치를 몇 번 만들 수 있는지 계산해, 총 몇 번의 iteration이 필요한지 계산
    self.iterations = sum(self.batches_per_class.values()) // self.N_way

    # 각 iteration에서 선택할 N_way개의 클래스를 저장 / 각 클래스에서 생성 가능한 배치 수 만큼 클래스를 반복하여 리스트에 추가
    self.class_list = [c for c in self.classes for _ in range(self.batches_per_class[c])]

    # True일 경우 shuffle_data를 호출하여 클래스 리스틀 한 번 Shuffle
    if shuffle_once or self.shuffle:
      self.shuffle_data()
    
    # False일 경우 = 데이터셋을 test 할 경우를 의미
    # 클래스를 섞는 대신, 정해진 순서대로 iteration을 진행해야 하므로, 이를 위해 인덱스 리스트를 계산해 클래스 리스트를 순서대로 정렬(np.argsort(sort_list))
    else:
      sort_idxs = [i + p * self.num_classes for i, c in enumerate(self.classes) for p in range(self.batches_per_class[c])]
      self.class_list = np.array(self.class_list)[np.argsort(sort_idxs)].tolist()
    ######################################################################################################

  def shuffle_data(self):  # 각 배치에서 다른 클래스의 데이터를 섞어 overfitting을 방지하고 학습을 일반화 시킴
    
    # 클래스 당 예제별 shuffle 
    for c in self.classes:      
      # indices_per_class[c]에서 원소의 인덱스를 섞는다
      perm = torch.randperm(self.indices_per_class[c].shape[0])
      
      # 위에서 섞인 perm의 인덱스 순서에 따라 indices_per_class[c]의 원소들을 섞는다
      self.indices_per_class[c] = self.indices_per_class[c][perm]
    
    # 모든 클래스를 포함한 classes_list의 순서를 랜덤하게 섞는다
    random.shuffle(self.class_list)

  def __iter__(self):
    
    # Shuffle data
    if self.shuffle:
      self.shuffle_data()

    #################### few-shot 배치 샘플 ####################
    # 딕셔너리의 기본값을 0으로 설정
    start_index = defaultdict(int)
    
    for i in range(self.iterations):
      # N_way의 개수 만큼 클래스를 class_batch에 할당
      class_batch = self.class_list[i * self.N_way : (i + 1) * self.N_way]
      index_batch = []
      for c in class_batch:
        # 클래스 c 에서 K-shot 만큼 인덱스를 선택하여 index_batch에 추가
        index_batch.extend(self.indices_per_class[c][start_index[c] : start_index[c] + self.K_shot])
        # 다음 반복을 위해 K-shot 만큼 건너뛰어 선택하도록
        start_index[c] += self.K_shot
      
      # Query_set을 포함하도록 선택하였다면, sup_set과 qry_set을 번갈아 가면서 배치
      if self.include_query:
        index_batch = index_batch[::2] + index_batch[1::2]

      # minibatch를 구성하는 데이터 샘플들의 인덱스 리스트 반환
      yield index_batch
    ###########################################################

  def __len__(self):
    return self.iterations

In [24]:
N_WAY = args.N_WAY
K_SHOT = args.K_SHOT
train_data_loader = torch.utils.data.DataLoader(train_set,
                                    batch_sampler=FewShotBatchSampler(train_set.label,
                                                                      include_query=True,
                                                                      N_way=N_WAY,
                                                                      K_shot=K_SHOT,
                                                                      shuffle=True),
                                    num_workers=1)
val_data_loader = torch.utils.data.DataLoader(valid_set,
                                  batch_sampler=FewShotBatchSampler(valid_set.label,
                                                                    include_query=True,
                                                                    N_way=N_WAY,
                                                                    K_shot=K_SHOT,
                                                                    shuffle=False,
                                                                    shuffle_once=True),
                                  num_workers=1)

In [25]:
def split_batch(imgs, targets):
    support_imgs, query_imgs = imgs.chunk(2, dim=0)
    support_targets, query_targets = targets.chunk(2, dim=0)
    return support_imgs, query_imgs, support_targets, query_targets

- sup_set과 qry_set 에는 동일한 5개의 클래스가 존재하지만 예제는 다르게 나타난다.

- 모델은 sup_set 및 해당 레이블에서 학습하여 qry_set의 예를 분류하는 task를 수행하게 된다. 

---
# 5. Prototype Networks
---

In [26]:
class BERT(nn.Module):
	def __init__(self,  n_input = 768, n_output = 128, bert_model = 'bert-base-uncased'):
		super(BERT,self).__init__()
		self.bert = BertModel.from_pretrained(bert_model)

	def forward(self, input_ids, attention_mask, token_type_ids):
		all_hidden_layers, _ = self.bert(input_ids = input_ids, 
                                     attention_mask = attention_mask,
                                     token_type_ids = token_type_ids, return_dict = False)
		cls_hn = all_hidden_layers[:,0,:]
		return cls_hn

In [27]:
class ProtoNet(pl.LightningModule):

  def __init__(self, lr):
    super().__init__()

    self.model = BERT()
    self.save_hyperparameters()
    self.lr = lr

  def configure_optimizers(self):
    optimizer = optim.AdamW(self.model.parameters(), lr = self.lr)
    scheduler = optim.lr_scheduler.MultiStepLR(
          optimizer, milestones = [20, 40], gamma = 0.1)
    
    return [optimizer], [scheduler]

  ################# few-shot classification에서 사용될 Prototype을 계산하는 함수 #################
  @staticmethod
  def calculate_prototypes(features, targets):
    # 정답 레이블에서 고유한 클래스 레이블을 찾아 리스트에 저장 및 정렬
    classes, _ = torch.unique(targets).sort()
    # 프로토타입을 저장할 빈 리스트
    prototypes = []
    for i in classes : # 클래스 라벨 i를 반복
      # 현재 클래스 i에 속하는 샘플들의 특징 벡터를 선택해 평균을 계산
      p = features[torch.where(targets == i)[0]].mean(dim = 0)
      prototypes.append(p)
    
    # prototypes 리스트를 tensor로 변환하여 반환 (dim = 0는 행으로 스택)
    prototypes = torch.stack(prototypes, dim = 0)

    return prototypes, classes
  
  ############## 새로운 샘플을 프로토타입으로 분류하고 분류 오류를 반환하는 함수 ##############
  def classify_feats(self, prototypes, classes, feats, targets):

    # 두 벡터 사이의 유클리드 거리(2- 제곱 거리)를 계산
    dist = torch.pow(prototypes[None, :] - feats[:, None], 2).sum(dim = 2)
    '''prototype과 feats는 각각 K x D 크기의 Tensor
       k = 클래스의 수 / D = 특징 벡터
       Prototype[None, :] .> [1 x K x D] 
       feats[:, None] -> [N x 1 x D]
       두 tensor를 뺀 결과는 [N x K x D] 이며 제곱을 하고 마지막 차원을 따라 더하면 [N x K]의 형태가 나옴'''
  
    # 클래스에 대한 로그 확률을 계산 
    preds = F.log_softmax( - dist, dim = 1)

    # 각 행에서 가장 높은 값을 가지는 열의 인덱스를 찾아 반환
    labels = (classes[None, :] == targets[:, None]).long().argmax(dim=1)
    '''classes = 모든 클래스 ID를 포함하는 tensor
       targets = 각 샘플의 실제 클래스 ID를 포함하는 tensor
       첫 번째 차원에 대해 브로드캐스팅하여 [N x K]의 tensor'''
  
    # preds과 labels 값의 분류 정확도 계산
    acc = (preds.argmax(dim=1) == labels).float().mean()

    return preds, labels, acc


####################3# 잠시 보류 ####################3#
  # def classify_feats(self, prototypes, classes, feats, targets):
  #   # 두 벡터 사이의 유클리드 거리(2- 제곱 거리)를 계산
  #   dist = torch.pow(prototypes[:, None] - feats, 2).sum(dim = 2)
  #   # 클래스에 대한 로그 확률을 계산 
  #   preds = F.log_softmax( - dist, dim = 1)

  #   # 각 행에서 가장 높은 값을 가지는 열의 인덱스를 찾아 반환
  #   labels = (classes[:, None] == targets[None, :]).long().argmax(dim=1)
    
  #   # preds과 labels 값의 분류 정확도 계산
  #   acc = (preds.argmax(dim=1) == labels).float().mean()

  #   return preds, labels, acc
####################3#####################3#####################3#

  ################### 현재 모델과 배치를 기반으로 손실을 계산하는 함수 ###################
  def calculate_loss(self, batch, mode):
    
    text, label = batch

    # 모델을 통해 인코딩된 이미지 특성 벡터 생성 (sup_set과 qry_set)
    features = self.model(text['input_ids'].squeeze(1),
                          text['attention_mask'].squeeze(1),
                          text['token_type_ids'].squeeze(1))

    # sup_set의 특성 벡터, qry_set의 특성 벡터, sup_set의 레이블, qry_set의 레이블 계산
    support_feats, query_feats, support_targets, query_targets = split_batch(features, label)

    # sup_set에서 각 클래스의 Prototype을 계산
    prototypes, classes = ProtoNet.calculate_prototypes(support_feats, support_targets)

    # qry_set에 대해서 Prototype을 이용해 분류를 수행하고 예측 결과, 실제 레이블, 정확도 계산
    preds, labels, acc = self.classify_feats(prototypes, classes, query_feats, query_targets)

    # loss 계산
    loss = F.cross_entropy(preds, labels)

    # 모델의 log 저장
    self.log(f"{mode}_loss", loss)
    self.log(f"{mode}_acc", acc)
    
    return loss

  def training_step(self, batch, batch_idx):
      return self.calculate_loss(batch, mode="train")

  def validation_step(self, batch, batch_idx):
      _ = self.calculate_loss(batch, mode="val")

---
# 6. Train
----

In [28]:
CHECKPOINT_PATH = '/content/drive/MyDrive/2.Study/Few-Shot/'

def train_model(model_class, train_loader, val_loader, **kwargs):

  trainer = pl.Trainer(default_root_dir = os.path.join(CHECKPOINT_PATH, model_class.__name__),
                       accelerator = "gpu" if str(device).startswith("cuda") else "cpu",
                       devices = 1,
                       max_epochs = args.epoch,
                       callbacks = [ModelCheckpoint(save_weights_only=True, mode="max", monitor="val_acc"),
                                    LearningRateMonitor("epoch")],
                       enable_progress_bar = True)
  '''default_root_dir = 체크포인트 파일이 저장될 경로
     accelerator = CPU 또는 GPU
     devices = 학습에 사용할 디바이스 개수
     max_epochs = 학습 에폭 수
     callbacks = 학습 도중에 호출될 콜백 목록, 체크포인트 저장, 모니터링 여부 등
     enable_progress_bar = 진행 상황 표시 여부'''
  trainer.logger._default_hp_metric = None

  # 사전에 훈련한 체크 포인트가 있는지 확인하고 있다면 연결, 없다면 새로 학습
  pretrained_filename = os.path.join(CHECKPOINT_PATH, model_class.__name__  + '.ckpt')

  if os.path.isfile(pretrained_filename):
    print(f"Found pretrained model at {pretrained_filename}, loading...")
    # 자동적으로 모델 및 하이퍼파라미터 로드
    model = model_class.load_from_checkpoint(pretrained_filename)
  else:
    # 랜덤 시드를 고정하여 재현성 보장
    pl.seed_everything(42)
    
    # 주어진 모델 클래스와 인자로 부터 모델 객체를 생성
    model = model_class(**kwargs)
    
    # 모델 학습 
    trainer.fit(model, train_loader, val_loader)

    # 학습이 완료된 후 체크포인트 콜백에서 가장 좋은 성능을 보인 모델의 체크포인트 파일을 로드하요 모델 생성
    model = model_class.load_from_checkpoint(
              trainer.checkpoint_callback.best_model_path)
    
  return model

- 학습 진행

In [29]:
protonet_model = train_model(ProtoNet,
                             lr=args.lr,
                             train_loader=train_data_loader,
                             val_loader=val_data_loader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:lightning_fabric.utilities.seed:Global seed set to 42
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- 

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

IsADirectoryError: ignored

In [ ]:
%tensorboard --logdir ../content/drive/MyDrive/2.Study/Few-Shot/ProtoNet

---
# 7. Test
----

In [ ]:
test_set = BertDataset(test_data)

In [ ]:
@torch.no_grad()
def test_proto_net(model, dataset, data_feats=None, k_shot = 4):
    '''model = 사전 학습된 ProtoNet Model
       dataset = test_dataset 사용
       data_feats = 이미지 특징 데이터
       K_shot = 샷의 수'''
    
    model = model.to(device)
    model.eval()

    # 데이터셋에서 고유한 클래스 수 계산
    num_classes = dataset.label.unique().shape[0]   # num_classes = 6
    
    # 각 클래스 당 이미지 샘플 수 계산
    exmps_per_class = dataset.label.shape[0]//num_classes # exmps_per_class = 31 


    if data_feats is None:

        dataloader = torch.utils.data.DataLoader(dataset, batch_size=128, num_workers=1, shuffle=False, drop_last=False)
        # 추출된 이미지 특징을 저장할 리스트
        img_features = []
        # 추출된 이미지 라벨을 저장할 리스트
        img_targets = []
        for imgs, targets in tqdm(dataloader):
            imgs = imgs.to(device)
            feats = model.model(imgs['input_ids'].squeeze(1), # feats = [batch_size, embedding_vector] /
                                imgs['attention_mask'].squeeze(1),
                                imgs['token_type_ids'].squeeze(1))
            img_features.append(feats.detach().cpu())         # img_features = [batch_size, embedding_vector] * n 의 리스트 형태 
            img_targets.append(targets)                       # img_targets = [batch_size] * n 의 리스트 형태

        # 리스트에 있는 tensor를 하나의 tensor로 결합 (dim = 0 은 첫 번째 차원으로 결합)
        img_features = torch.cat(img_features, dim=0)         
        img_targets = torch.cat(img_targets, dim=0)           

        img_targets, sort_idx = img_targets.sort()
        # tensor를 2차원 형태로 재구성 -> 차원 변경 [exmps_per_class x num_classes]
        img_targets = img_targets.reshape(num_classes, exmps_per_class).transpose(0, 1)
        # tensor를 sort_idx를 사용하여 정렬하고 3차원 tensor로 재구성 -> 차원 변경 [exmps_per_class x num_classes x feat_dim]
        img_features = img_features[sort_idx].reshape(num_classes, exmps_per_class, -1).transpose(0, 1)

    else:
        img_features, img_targets = data_feats

    '''아래 첫 번째 for문 에서는 k-shot classification을 사용해 모든 클래스의 프로토타입을 계산
       이를 위해, 각 클래스에서 k 개의 샘플을 선택하고, 특징과 타겟을 추출 하고 
       calculate_prototype을 사용해 각 클래스의 프로토타입과 클래스를 계산
       
       두 번째 for문에서는 k-shot classification을 사용해 계산된 프로토타입을 사용하여 다른 모든 샘플을 분류
       이를 위해, 각 샘플이 어떤 클래스에 속하는지 예측하고, 실제 타겟과 비교해여 정확도를 계산
       
       요약하자면, 첫 번째 for문에서는 각 클래스의 프로토타입을 계산
                두 번째 for문 에서는 프로토타입을 사용해 모든 샘플을 분류한 후 정확도를 계산'''

    accuracies = []
    # K_shot개씩 샘플을 뽑아 프로토 타입을 계산
    for k_idx in tqdm(range(0, img_features.shape[0], k_shot)):
        
        # K_shot개씩 샘플을 뽑아 샘플의 특징과 타겟을 저장
        k_img_feats, k_targets = img_features[k_idx:k_idx+k_shot].flatten(0,1), img_targets[k_idx:k_idx+k_shot].flatten(0,1)
        # 프로토타입을 계산
        prototypes, proto_classes = model.calculate_prototypes(k_img_feats, k_targets)

        batch_acc = 0
        # 모든 샘플을 돌면서 정확도를 계산
        for e_idx in range(0, img_features.shape[0], k_shot):
            # 만약 프로토타입에서 사용한 샘플이라면 건너뛴다.
            if k_idx == e_idx:
                continue
            # 현재 샘플에서 K_shot개씩 샘플을 뽑아 샘플의 특징과 타겟을 저장
            e_img_feats, e_targets = img_features[e_idx:e_idx+k_shot].flatten(0,1), img_targets[e_idx:e_idx+k_shot].flatten(0,1)

            # e_img_feats를 이용해 분류하고 정확도를 계산
            _, _, acc = model.classify_feats(prototypes, proto_classes, e_img_feats, e_targets)

            batch_acc += acc.item()
        batch_acc /= img_features.shape[0] // k_shot - 1
        accuracies.append(batch_acc)

    # (평균, 표준편차) (feature, targets)
    return (mean(accuracies), stdev(accuracies)), (img_features, img_targets)

In [ ]:
protonet_accuracies = dict()
data_feats = None
# for k in [2, 4, 6, 8, 10, 12, 14, 16]:
for k in range(2,15):
    protonet_accuracies[k], data_feats = test_proto_net(protonet_model, test_set, data_feats=data_feats, k_shot=k)
    print(f"Accuracy for k={k}: {100.0*protonet_accuracies[k][0]:4.2f}% (+-{100*protonet_accuracies[k][1]:4.2f}%)", '\n')

# 이미지 당 샘플의 수가 31개 이기 때문에 k-shot의 크기가 16를 넘어가면 zero-division error가 발생한다.
# batch_acc /= img_features.shape[0] // k_shot - 1 이 수식에서 0에 가까운 수가 나오기 때문에 발생하는 오류인것 같다.

In [ ]:
def plot_few_shot(acc_dict, name, color=None, ax=None):
    sns.set()
    if ax is None:
        fig, ax = plt.subplots(1,1,figsize=(5,3))
    ks = sorted(list(acc_dict.keys()))
    mean_accs = [acc_dict[k][0] for k in ks]
    std_accs = [acc_dict[k][1] for k in ks]
    ax.plot(ks, mean_accs, marker='o', markeredgecolor='k', markersize=6, label=name, color=color)
    ax.fill_between(ks, [m-s for m,s in zip(mean_accs, std_accs)], [m+s for m,s in zip(mean_accs, std_accs)], alpha=0.2, color=color)
    ax.set_xticks(ks)
    ax.set_xlim([ks[0]-1, ks[-1]+1])
    ax.set_xlabel("Number of shots per class", weight='bold')
    ax.set_ylabel("Accuracy", weight='bold')
    if len(ax.get_title()) == 0:
        ax.set_title("Few-Shot Performance " + name, weight='bold')
    else:
        ax.set_title(ax.get_title() + " and " + name, weight='bold')
    ax.legend()
    return ax

In [ ]:
ax = plot_few_shot(protonet_accuracies, name="ProtoNet", color="C1")
plt.show()
plt.close()

---
# 8. Another Dataset Adapt
---

In [ ]:
# test
text = pd.read_csv('./SNIPS/test/seq.in', names = ['text'])
label = pd.read_csv('./SNIPS/test/label', names = ['label'])
another_data = pd.concat([text,label], axis = 1

In [ ]:
another_set = BertDataset(another_data)

In [ ]:
protonet_another_accuracies = dict()
data_feats = None
# for k in [2, 4, 6, 8, 10, 12, 14, 16]:
for k in range(2,15):
    protonet_another_accuracies[k], data_feats = test_proto_net(protonet_model, another_set, data_feats=data_feats, k_shot=k)
    print(f"Accuracy for k={k}: {100.0*protonet_accuracies[k][0]:4.2f}% (+-{100*protonet_accuracies[k][1]:4.2f}%)", '\n')

# 이미지 당 샘플의 수가 31개 이기 때문에 k-shot의 크기가 16를 넘어가면 zero-division error가 발생한다.
# batch_acc /= img_features.shape[0] // k_shot - 1 이 수식에서 0에 가까운 수가 나오기 때문에 발생하는 오류인것 같다.